In [1]:
import pandas as pd

In [4]:
df = pd.read_excel(r'/content/drive/MyDrive/Colab Notebooks/上證指數.xlsx')
df

,名稱,代碼,日期,開盤,最高,最低,收盤,漲跌
0,上證指數,980001,2017-01-03,3105.31,3136.46,3105.31,3135.92,1
1,上證指數,980001,2017-01-04,3133.79,3160.10,3130.11,3158.79,1
2,上證指數,980001,2017-01-05,3157.91,3168.50,3154.28,3165.41,1
3,上證指數,980001,2017-01-06,3163.78,3172.03,3153.03,3154.32,0
4,上證指數,980001,2017-01-09,3148.53,3173.14,3147.74,3171.24,1
...,...,...,...,...,...,...,...,...
177,上證指數,980001,2017-09-21,3364.70,3377.88,3356.88,3357.81,0
178,上證指數,980001,2017-09-22,3347.16,3356.45,3334.98,3352.53,0
179,上證指數,980001,2017-09-25,3344.59,3350.96,3334.94,3341.55,0
180,上證指數,980001,2017-09-26,3336.35,3347.16,3332.60,3343.58,1


In [5]:
import torch
import torch.utils.data as Data
import numpy as np

In [38]:
class MyDataset(Data.Dataset):
  def __init__(self, file_path, seq_length, train=True, transform=None):
    df = pd.read_excel(file_path)
    df_len = len(df)-1
    df_len -= (df_len%seq_length)
    toto_size = df_len//seq_length
    train_size = int(toto_size*0.7)

    f = df.iloc[:(train_size*seq_length),3:7]
    l = df.iloc[1:(train_size*seq_length)+1,7:]
    data_size = train_size

    if train == False:
      f = df.iloc[(train_size*seq_length):df_len,3:7]
      l = df.iloc[(train_size*seq_length)+1:df_len+1,7:]
      data_size = toto_size-train_size

    self.features = np.array(f).astype("float32").reshape(data_size,seq_length,4)
    self.labels = np.array(l).astype("int").reshape(data_size,seq_length,1)
    self.transform = transform

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()

    X = self.features[idx]
    Y = self.labels[idx]

    if self.transform:
      X = self.transform(X)
      Y = self.transform(Y)

    return X, Y

In [30]:
def toTensor(x):
  return torch.tensor(x)

In [39]:
file_path = r'/content/drive/MyDrive/Colab Notebooks/上證指數.xlsx'
train_dataset = MyDataset(file_path=file_path, seq_length=5, train=True, transform=toTensor)
test_dataset = MyDataset(file_path=file_path, seq_length=5, train=False, transform=toTensor)

In [41]:
featurs, labes, = train_dataset[2:9]
print(featurs.shape)
print(labes.shape)
featurs

torch.Size([7, 5, 4])
torch.Size([7, 5, 1])


tensor([[[3087.0300, 3108.9099, 3072.3401, 3108.7700],
         [3104.7700, 3123.7200, 3098.5901, 3113.0100],
         [3104.9700, 3115.7800, 3094.0100, 3101.3000],
         [3095.8201, 3125.6599, 3095.2100, 3123.1399],
         [3125.4199, 3145.8401, 3125.4199, 3136.7700]],

        [[3134.5901, 3149.5300, 3131.2200, 3142.5500],
         [3137.6499, 3151.4700, 3133.1899, 3149.5500],
         [3149.2200, 3163.1001, 3148.9099, 3159.1699],
         [3160.0801, 3162.6799, 3136.0100, 3140.1699],
         [3143.0901, 3158.8401, 3135.3899, 3156.9800]],

        [[3154.3999, 3159.5400, 3140.0400, 3153.0901],
         [3148.0901, 3167.4500, 3132.0300, 3166.9800],
         [3164.6899, 3186.8401, 3162.5701, 3183.1799],
         [3183.0100, 3205.0500, 3182.8000, 3196.7000],
         [3198.9900, 3219.4099, 3198.9900, 3216.8401]],

        [[3216.1399, 3219.3999, 3205.2900, 3217.9299],
         [3215.4600, 3236.0000, 3206.5601, 3212.9900],
         [3210.3601, 3230.2800, 3207.7900, 3229.6201],
    

In [42]:
train_loader = Data.DataLoader(dataset=train_dataset, batch_size=5, shuffle=True)
test_loader = Data.DataLoader(dataset=test_dataset, batch_size=5, shuffle=False)

In [43]:
for i, (seqs, labels) in enumerate(train_loader):
  print(seqs.shape)
seqs

torch.Size([5, 5, 4])
torch.Size([5, 5, 4])
torch.Size([5, 5, 4])
torch.Size([5, 5, 4])
torch.Size([5, 5, 4])


tensor([[[3242.2200, 3254.3401, 3239.8799, 3253.3301],
         [3252.6899, 3261.3799, 3243.8401, 3261.2200],
         [3258.8301, 3264.0801, 3236.3501, 3251.3799],
         [3246.8601, 3253.9600, 3233.5300, 3253.4299],
         [3249.1899, 3251.6499, 3224.0901, 3228.6599]],

        [[3134.5901, 3149.5300, 3131.2200, 3142.5500],
         [3137.6499, 3151.4700, 3133.1899, 3149.5500],
         [3149.2200, 3163.1001, 3148.9099, 3159.1699],
         [3160.0801, 3162.6799, 3136.0100, 3140.1699],
         [3143.0901, 3158.8401, 3135.3899, 3156.9800]],

        [[3086.7100, 3095.4800, 3081.2800, 3090.6299],
         [3087.1699, 3103.9399, 3063.1499, 3075.6799],
         [3069.3899, 3084.2400, 3050.8401, 3061.9500],
         [3047.5701, 3064.8101, 3022.3000, 3064.0801],
         [3055.3401, 3114.6599, 3052.8301, 3107.8301]],

        [[3148.0200, 3150.4600, 3134.6101, 3140.0100],
         [3148.9900, 3157.0300, 3132.6201, 3156.2100],
         [3152.2400, 3186.9800, 3146.6399, 3147.4500],
    

In [44]:
input_size = 4
hidden_size = 32
num_layers = 2
num_class = 1

In [45]:
import torch.nn as nn
from torch.autograd import Variable

In [49]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_class):
    super(RNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.normal = nn.BatchNorm1d(5, affine=True)
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size, num_class)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
    c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
    out = self.normal(x)
    out, _ = self.lstm(out, (h0, c0))
    out = self.fc(out)
    out = self.sigmoid(out)
    return out


In [51]:
rnn = RNN(input_size, hidden_size, num_layers, num_class)
rnn

RNN(
  (normal): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(4, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [53]:
output = rnn(seqs)
output

tensor([[[0.4775],
         [0.4767],
         [0.4766],
         [0.4767],
         [0.4769]],

        [[0.4777],
         [0.4776],
         [0.4780],
         [0.4786],
         [0.4790]],

        [[0.4781],
         [0.4787],
         [0.4800],
         [0.4816],
         [0.4826]],

        [[0.4778],
         [0.4778],
         [0.4779],
         [0.4785],
         [0.4788]],

        [[0.4781],
         [0.4783],
         [0.4789],
         [0.4792],
         [0.4793]]], grad_fn=<SigmoidBackward0>)

In [54]:
import torch.optim as opt

In [58]:
loss_fn = nn.BCELoss()
optimizer = opt.Adam(rnn.parameters(), lr=0.001)

In [59]:
for epoch in range(800):
  for i, (seqs, labels) in enumerate(train_loader):
    seqs = Variable(seqs)
    labels = Variable(labels)
    optimizer.zero_grad()
    outputs = rnn(seqs)
    loss = loss_fn(outputs, labels.float())
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
      print("Epoch: %d, batch: %d, loss: %1.5f" % (epoch, i, loss.item()))

Epoch: 99, batch: 0, loss: 0.64291
Epoch: 99, batch: 1, loss: 0.66656
Epoch: 99, batch: 2, loss: 0.71470
Epoch: 99, batch: 3, loss: 0.61379
Epoch: 99, batch: 4, loss: 0.64846
Epoch: 199, batch: 0, loss: 0.59001
Epoch: 199, batch: 1, loss: 0.67518
Epoch: 199, batch: 2, loss: 0.71121
Epoch: 199, batch: 3, loss: 0.62382
Epoch: 199, batch: 4, loss: 0.59379
Epoch: 299, batch: 0, loss: 0.63493
Epoch: 299, batch: 1, loss: 0.69905
Epoch: 299, batch: 2, loss: 0.57856
Epoch: 299, batch: 3, loss: 0.66893
Epoch: 299, batch: 4, loss: 0.62628
Epoch: 399, batch: 0, loss: 0.57470
Epoch: 399, batch: 1, loss: 0.65987
Epoch: 399, batch: 2, loss: 0.50177
Epoch: 399, batch: 3, loss: 0.75319
Epoch: 399, batch: 4, loss: 0.66265
Epoch: 499, batch: 0, loss: 0.64974
Epoch: 499, batch: 1, loss: 0.56925
Epoch: 499, batch: 2, loss: 0.65802
Epoch: 499, batch: 3, loss: 0.56239
Epoch: 499, batch: 4, loss: 0.56207
Epoch: 599, batch: 0, loss: 0.60872
Epoch: 599, batch: 1, loss: 0.56189
Epoch: 599, batch: 2, loss: 0.527

In [70]:
correct = 0
total = 0
for seqs, labels in test_loader:
  seqs = Variable(seqs)
  outputs = rnn(seqs)
  preducted = outputs.gt(0.5)
  total += (labels.size(0)*labels.size(1))
  correct += (preducted.int() == labels).sum()

print("Acc: %.3f%%"%(100.0 * float(correct) / float(total)))

Acc: 63.636%
